In [1]:
import os
import pandas as pd

DIR_DATA = os.path.join("..", "data")
PATH_DATA_TRAIN = os.path.join(DIR_DATA, "train.csv")
PATH_DATA_TEST = os.path.join(DIR_DATA, "test.csv")

In [2]:
df_train = pd.read_csv(PATH_DATA_TRAIN)

df_train_cleaned = df_train.copy()
df_train_cleaned['text'] = df_train_cleaned['text'].apply(lambda string: string.lower())
df_train_cleaned['text'] = df_train_cleaned['text'].replace(to_replace=r'https?:\/\/.*[\r\n]*|[^\w\s]', value = " ", regex=True)
df_train_cleaned['text'] = df_train_cleaned['text'].replace(to_replace=r'[\s]+', value=' ', regex=True)

In [21]:
from collections import defaultdict
import math
import sys
import time
from multiprocessing import Pool, Manager, Process
import multiprocessing
from markov_helper import count_ngrams, progress_monitor

MARKOV_SMOOTHING_WEIGHT = 1
MARKOV_NUM_CHARS = 95
MARKOV_MULTITHREAD_NUM_WORKERS = multiprocessing.cpu_count()

class MarkovModel:
    order: int
    ngram_freqs: list[dict]
    totals: list[int]
    __pool = Pool(processes=MARKOV_MULTITHREAD_NUM_WORKERS)
    __manager = Manager()

    def __init__(self, order: int, sequences: list = [], multithread: bool = True, verbose: bool = False):
        assert(order >= 0)


        self.order = order
        self.ngram_freqs = [defaultdict(int)] * (order + 1)
        self.totals = [0] * (order + 1)

        if (len(sequences) > 0):
            if isinstance(sequences[0], str):
                sequences = list(map(lambda string: list(string), sequences))

            start = time.time()

            if (multithread):
                processes = [None] * (order + 1)  
                progresses = [None] * (order + 1)

                try:
                    for n in range(order + 1):
                        progresses[n] = MarkovModel.__manager.Value('i', 0)
                        process = MarkovModel.__pool.apply_async(count_ngrams, args=(sequences, n, progresses[n]))
                        processes[n] = process   

                    if verbose:
                        monitor = Process(target=progress_monitor, args=(progresses, order + 1, len(sequences)))
                        monitor.start()
                    
                    for n in range(len(processes)):
                        (total, frequencies) = processes[n].get()
                        self.totals[n] = total
                        self.ngram_freqs[n] = frequencies
                    
                except Exception as e:
                    MarkovModel.__pool.terminate()
                    MarkovModel.__pool = Pool(processes=MARKOV_MULTITHREAD_NUM_WORKERS)
                    raise e
                finally:
                    if verbose:
                        monitor.terminate()
                    
            else:
                    i = 0
                    for seq in sequences:
                        i += 1
                        print(f'Processing sequence {i:>{math.floor(math.log(len(sequences), 10)) + 1}}/{len(sequences)} for {order} orders', end='\r')
                        for n in range(0, order + 1):
                            for pos in range(len(seq) - n):
                                ngram = seq[pos:pos+n+1]
                                self.add_ngram_occurrence(ngram)

            duration = time.time() - start
            if verbose:
                print(f'Order {order} Markov Model of {len(sequences)} sequences built in {duration:.3f} seconds!')

    def __str__(self):
        return f'{self.order}-Order Markov NGram Object - Totals: {self.totals}'
    
    def __check_len(self, l):
        if l > self.order + 1 or l == 0:
            raise ValueError(f'Invalid NGram - Length of ngram must be between 1 and {self.order + 1} for an order {self.order} NGrams object (is {l})')

    @classmethod
    def __tuple_from_list(cls, l: list):
        t = (l[0],)
        for item in l[1:]:
            t += (item,)

        return t
    
    @classmethod
    def __fix_type(cls, a):
        if isinstance(a, str):
            return list(a)
        elif isinstance(a, list):
            return a
        else:
            raise TypeError(f'Argument must be of type string or list (is {type(a)})')

    def add_ngram_occurrence(self, ngram):
        self.__check_len(len(ngram))
        ngram = MarkovModel.__fix_type(ngram)
        self.ngram_freqs[len(ngram) - 1][MarkovModel.__tuple_from_list(ngram)] += 1
        self.totals[len(ngram) - 1] += 1

    def get_total_occurrences(self, ngram):
        self.__check_len(len(ngram))
        ngram = MarkovModel.__fix_type(ngram)
        return self.ngram_freqs[len(ngram) - 1][MarkovModel.__tuple_from_list(ngram)]
    
    def get_log_cond_prob(self, ngram):
        self.__check_len(len(ngram))
        ngram = MarkovModel.__fix_type(ngram)

        top = self.get_total_occurrences(ngram) + MARKOV_SMOOTHING_WEIGHT
        bot = self.totals[0] if (len(ngram) == 1) else self.get_total_occurrences(ngram[:-1])
        bot += MARKOV_SMOOTHING_WEIGHT * ((len(ngram) - 1) ** MARKOV_NUM_CHARS)
        
        return math.log(top / bot) if top > 0 else sys.minint
    
    def get_log_sum_prob(self, seq: str):
        if (len(seq) == 0):
            raise ValueError(f'Invalid Sequence - Cannot get probability for an empty list')

        start = 0
        end = 1
        sum = 0

        while (end <= len(seq)):
            sum += self.get_log_cond_prob(seq[start:end])
            end += 1

            # ngrams can be at most order + 1 characters long:
            if (end - start > self.order + 1):
                start += 1
        return sum


In [14]:
texts_pos = (df_train.loc[df_train['target'] == 1])['text'].to_list()
texts_neg = (df_train.loc[df_train['target'] == 0])['text'].to_list()

MARKOV_ORDER = 10

model_pos = MarkovModel(MARKOV_ORDER, list(map(lambda x: list(x), texts_pos)), verbose=True)
model_neg = MarkovModel(MARKOV_ORDER, list(map(lambda x: list(x), texts_neg)), verbose=True)
model_pos2 = MarkovModel(MARKOV_ORDER, list(map(lambda x: list(x), texts_pos)), False)
model_neg2 = MarkovModel(MARKOV_ORDER, list(map(lambda x: list(x), texts_neg)), False)

def compare(sequence: str):
    pos1 = model_pos.get_log_sum_prob(list(sequence))
    pos2 = model_pos2.get_log_sum_prob(list(sequence))
    neg1 = model_neg.get_log_sum_prob(list(sequence))
    neg2 = model_neg2.get_log_sum_prob(list(sequence))

    print(f'Positives match: {pos1 == pos2}, Negatives match: {neg1 == neg2}')

compare("They should all die! All of them! Everything annihilated!")
compare("@sakuma_en If you pretend to feel a certain way the feeling can become genuine all by accident. -Hei (Darker than Black) #manga #anime")
compare("Shot 12 times. Found dead in cuffs after being involved in a car accident. Officers told ambulance not to treat him. https://t.co/MEUDJwaaNg")
compare("@NinaHoag - 'if you shred my Psych work our friendship would be annihilated")
compare("Why should a helicopter ambulance ride to transfer to a hospital 21 miles away cost $29800?")
compare("If you build an army of 100 lions and their leader is a dog in any fight the lions will die like a dog.")
compare("One Direction Is my pick for http://t.co/q2eBlOKeVE Fan Army #Directioners http://t.co/eNCmhz6y34 x1424")

Order 10 Markov Model of 3271 sequences built in 0.613 seconds!
Order 10 Markov Model of 4342 sequences built in 0.749 seconds!
Positives match: True, Negatives match: True
Positives match: True, Negatives match: True
Positives match: True, Negatives match: True
Positives match: True, Negatives match: True
Positives match: True, Negatives match: True
Positives match: True, Negatives match: True
Positives match: True, Negatives match: True


In [16]:
def predict(sequence: str):
    print(list(sequence))
    pos = model_pos.get_log_sum_prob(sequence)
    neg = model_neg.get_log_sum_prob(sequence)
    print(f'pos: {pos}, neg: {neg}')
    print(f'The sentence is likely ', end='')
    if (pos < neg):
        print('not ', end='')
    print(f'about a real disaster. (\'{sequence}\')')

predict("They should all die! All of them! Everything annihilated!")
predict("@sakuma_en If you pretend to feel a certain way the feeling can become genuine all by accident. -Hei (Darker than Black) #manga #anime")
predict("Shot 12 times. Found dead in cuffs after being involved in a car accident. Officers told ambulance not to treat him. https://t.co/MEUDJwaaN")
predict("@NinaHoag - 'if you shred my Psych work our friendship would be annihilated")
predict("Why should a helicopter ambulance ride to transfer to a hospital 21 miles away cost $29800?")
predict("If you build an army of 100 lions and their leader is a dog in any fight the lions will die like a dog.")
predict("One Direction Is my pick for http://t.co/q2eBlOKeVE Fan Army #Directioners http://t.co/eNCmhz6y34 x1424")


['T', 'h', 'e', 'y', ' ', 's', 'h', 'o', 'u', 'l', 'd', ' ', 'a', 'l', 'l', ' ', 'd', 'i', 'e', '!', ' ', 'A', 'l', 'l', ' ', 'o', 'f', ' ', 't', 'h', 'e', 'm', '!', ' ', 'E', 'v', 'e', 'r', 'y', 't', 'h', 'i', 'n', 'g', ' ', 'a', 'n', 'n', 'i', 'h', 'i', 'l', 'a', 't', 'e', 'd', '!']
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing type
fixing

In [17]:
class ConfusionMatrix:
    tp: int
    fp: int
    tn: int
    fn: int

    def __init__(self, tp: int = 0, tn: int = 0, fp: int = 0, fn: int = 0):
        self.tp = tp
        self.fp = fp
        self.tn = tn
        self.fn = fn

    def __str__(self):
        total = self.tp + self.fp + self.tn + self.fn
        return f'Total: {total:.1f}, TP: {self.tp:.1f}, TN: {self.tn:.1f}, FP: {self.fp:.1f}, FN: {self.fn:.1f}, Prc: {self.get_precision():.3f}, Rec. {self.get_recall():.3f}, F1: {self.get_f1():.3f}'
    
    def get_f1(self):
        return 2 * ((self.get_precision() * self.get_recall()) / (self.get_precision() + self.get_recall()))
    
    def get_precision(self):
        return self.tp / (self.tp + self.fp)

    def get_recall(self):
        return self.tp / (self.tp + self.fn) 

    def add(self, rhs: 'ConfusionMatrix'):
        self.tp += rhs.tp
        self.fp += rhs.fp
        self.tn += rhs.tn
        self.fn += rhs.fn

    def div(self, divisor):
        self.tp /= divisor
        self.fp /= divisor
        self.tn /= divisor
        self.fn /= divisor

    @classmethod
    def average(cls, matrices: list['ConfusionMatrix']):
        result = ConfusionMatrix()
        for cm in matrices:
            result.add(cm)
        result.div(len(matrices))

        return result
        

In [18]:
class MarkovClassifier:
    model_pos: MarkovModel
    model_neg: MarkovModel
    order: int

    def __init__(self, order: int, positives: list, negatives: list, verbose: bool = False):
        self.model_pos = MarkovModel(order, positives, multithread=True, verbose=verbose)
        self.model_neg = MarkovModel(order, negatives, multithread=True, verbose=verbose)
        self.order = order

    def classify(self, seq: list) -> bool:
        return self.model_pos.get_log_sum_prob(seq) > self.model_neg.get_log_sum_prob(seq)

    def eval(self, positives: list, negatives: list) -> ConfusionMatrix:

        res = ConfusionMatrix()

        for instance in positives:
            if self.classify(instance):
                res.tp += 1
            else:
                res.fn += 1

        for instance in negatives:
            if not self.classify(instance):
                res.tn += 1
            else:
                res.fp += 1
        
        return res
    

In [19]:
import regex as re

def get_crossval_sets(df: pd.DataFrame, num_sets: int):

    df_shuffled: pd.DataFrame = df.sample(frac=1)
    len_chunk = len(df) / num_sets

    sets = {}
    sets['train'] = [None] * num_sets
    sets['val'] = [None] * num_sets

    for i in range(num_sets):
        start = round(i * len_chunk)
        end = round((i + 1) * len_chunk)
        sets['val'][i] = df_shuffled.iloc[start:end,:]
        sets['train'][i] = df_shuffled[~df_shuffled.index.isin(sets['val'][i].index)]

    return sets

def get_wordlist(text: str) -> list[str]:
    cleaned = text.lower()
    cleaned = re.sub(r'https?:\/\/.*[\r\n]*|[^\w\s]', ' ', cleaned)
    return cleaned.split()

def get_instances(df: pd.DataFrame, target: int):
    return list(map(lambda string: string, (df.loc[df['target'] == target])['text'].to_list()))

In [22]:
NUM_SETS = 5

sets = get_crossval_sets(df_train_cleaned, NUM_SETS)

for order in range(0, 15):
    res = ConfusionMatrix()
    time_training = 0.0
    time_validation = 0.0

    for i in range(NUM_SETS):
        start = time.time()
        classifier = MarkovClassifier(order, get_instances(sets['train'][i], 1), get_instances(sets['train'][i], 0))
        time_training += time.time() - start
        start = time.time()
        cm = classifier.eval(get_instances(sets['val'][i], 1), get_instances(sets['val'][i], 0))
        time_validation += time.time() - start
        res.add(cm)

    res.div(len(sets['train']))
    print(f'Order {order} results: ', end='')
    print(res)
    print(f'Training: [Tot.: {time_training:5.3f}s, Avg.: {time_training / NUM_SETS:5.3f}s], Validation: [Tot.: {time_validation:5.3f}s, Avg.: {time_validation / NUM_SETS:5.3f}s]')

Order 0 results: Total: 1522.6, TP: 374.8, TN: 610.6, FP: 257.8, FN: 279.4, Prc: 0.592, Rec. 0.573, F1: 0.583
Training: [Tot.: 0.636s, Avg.: 0.127s], Validation: [Tot.: 1.048s, Avg.: 0.210s]
Order 1 results: Total: 1522.6, TP: 471.6, TN: 624.2, FP: 244.2, FN: 182.6, Prc: 0.659, Rec. 0.721, F1: 0.688
Training: [Tot.: 0.728s, Avg.: 0.146s], Validation: [Tot.: 1.607s, Avg.: 0.321s]
Order 2 results: Total: 1522.6, TP: 231.8, TN: 852.6, FP: 15.8, FN: 422.4, Prc: 0.936, Rec. 0.354, F1: 0.514
Training: [Tot.: 1.010s, Avg.: 0.202s], Validation: [Tot.: 1.934s, Avg.: 0.387s]
Order 3 results: Total: 1522.6, TP: 358.4, TN: 827.6, FP: 40.8, FN: 295.8, Prc: 0.898, Rec. 0.548, F1: 0.680
Training: [Tot.: 1.156s, Avg.: 0.231s], Validation: [Tot.: 2.161s, Avg.: 0.432s]
Order 4 results: Total: 1522.6, TP: 403.0, TN: 812.8, FP: 55.6, FN: 251.2, Prc: 0.879, Rec. 0.616, F1: 0.724
Training: [Tot.: 1.535s, Avg.: 0.307s], Validation: [Tot.: 2.439s, Avg.: 0.488s]
Order 5 results: Total: 1522.6, TP: 419.4, TN: 7

KeyboardInterrupt: 

In [16]:
df_test = pd.read_csv(PATH_DATA_TEST)

classifier = MarkovClassifier(6, get_instances(df_train, 1), get_instances(df_train, 0))
df_test['target'] = df_test['text'].apply(classifier.classify).astype(int)

display(df_test)

,id,keyword,location,text,target
0,0,NaN,NaN,Just happened a terrible car crash,1
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",1
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",1
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,1
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,1
...,...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,0
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...,1
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...,1
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...,1


In [17]:
df_submission = df_test[['id', 'target']]
display(df_submission)
df_submission.to_csv(os.path.join(DIR_DATA, 'submission.csv'), index=False)

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,0
3259,10865,1
3260,10868,1
3261,10874,1
